In [40]:
from database.models import Assets, AssetsHistory
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import tools.tools as tools
import requests
import json

### Inicialização do Banco SQLite

In [8]:
assets = Assets()
assets.drop_table()
assets.create_table()

assets_history = AssetsHistory()
assets_history.drop_table()
assets_history.create_table()

### Extração de Dados da API Assets e Inserção no Banco

In [38]:
#Config
coins = "bitcoin,ethereum,dogecoin"

#Extração
BASE_ASSET_URL = f"http://api.coincap.io/v2/assets?ids={coins}"
response = requests.get(BASE_ASSET_URL)
items = response.json()['data']

#Inserção
for item in items:
    assets.insert((item['symbol'], item['name']))

In [46]:
#Configs
today = date.today()
past_date = today - relativedelta(years=5)
dt_format = "%Y-%m-%d"

start_unixtime = tools.to_unixtime(past_date.strftime(dt_format))
end_unixtime = tools.to_unixtime(today.strftime(dt_format))

#Extração
BASE_HISTORY_URL = f'http://api.coincap.io/v2/assets/bitcoin/history?start={start_unixtime}&end={end_unixtime}&interval=d1'
response = requests.get(BASE_HISTORY_URL)

items = response.json()['data']

#Inserção
for item in items:
    assets_history.insert((
        float(item['priceUsd']), 
        item['date'][0:10],
        1))


### Extração dos Dados Formatados do Banco de Dados

### Gráficos